Make a copy of this notebook and rename using your USERID in the following format, 2017CSZ8058

Give editor access to keshavkolluru@gmail.com, vishalsaley114@gmail.com and kartikeya.badola@gmail.com


In [1]:
## DONT CHANGE THIS CELL 
# this is currently the same as dev.data.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt

--2021-10-25 18:04:10--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt [following]
--2021-10-25 18:04:11--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63639 (62K) [text/plain]
Saving to: ‘test.data.txt’

test.data.txt       100%[===================>]  62.15K  78.0KB/s    in 0.8s    

2021-10-25 18:04:13 (78.0 KB/s) - ‘test.data.txt’ saved [63639/63639]



In [2]:
## Replace with the right link that contains the zip file uploaded from the training
!gdown https://drive.google.com/uc?id=14bHku5JrgsIC3lvCuB1ni9TKExSse1ox

Downloading...
From: https://drive.google.com/uc?id=14bHku5JrgsIC3lvCuB1ni9TKExSse1ox
To: /content/BestModel.zip
100% 1.08G/1.08G [00:09<00:00, 110MB/s] 


In [3]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 2.9 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 25.8 MB/s 
     |████████████████████████████████| 3.3 MB 35.1 MB/s 
     |████████████████████████████████| 56 kB 4.6 MB/s 
     |████████████████████████████████| 596 kB 46.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 290 kB 5.5 MB/s 
     |████████████████████████████████| 243 kB 39.2 MB/s 
     |████████████████████████████████| 1.3 MB 42.2 MB/s 
     |████████████████████████████████| 125 kB 42.9 MB/s 
     |████████████████████████████████| 271 kB 51.4 MB/s 
     |████████████████████████████████| 160 kB 47.3 MB/s 


In [4]:
!unzip BestModel.zip

Archive:  BestModel.zip
   creating: content/models/checkpoint-1358/
  inflating: content/models/checkpoint-1358/optimizer.pt  
  inflating: content/models/checkpoint-1358/scheduler.pt  
  inflating: content/models/checkpoint-1358/config.json  
  inflating: content/models/checkpoint-1358/rng_state.pth  
  inflating: content/models/checkpoint-1358/trainer_state.json  
  inflating: content/models/checkpoint-1358/pytorch_model.bin  
  inflating: content/models/checkpoint-1358/training_args.bin  


In [5]:
## Import relevant packages
import torch
torch.cuda.empty_cache()

import pandas as pd

import random
import numpy as np
from torch.utils.data import Dataset

from transformers import BertTokenizer, BertModel, BertForSequenceClassification, Trainer, TrainingArguments

from datasets import load_metric

rs = 0
torch.manual_seed(rs)
random.seed(rs)
np.random.seed(rs)
torch.backends.cudnn.deterministic = True


In [6]:
def prepareTestDataset():
    x_test = pd.read_csv('/content/test.data.txt', sep = '\t',header = None, names = ["word","pos","P1P2" , "sentence1", "sentence2"])
    b = x_test["P1P2"].str.split(pat = "-",expand = True)
    b.columns = ['P1','P2']
    x_test = x_test.join(b)
    x_test["sep"] = " [SEP] "
    x_test["sentence1"] = x_test["sentence1"].apply(lambda x: x.lower())
    x_test["sentence2"] = x_test["sentence2"].apply(lambda x: x.lower())
    x_test['word'] = x_test['word'].apply(lambda x: x.lower())
    # x_test["Sent"] = x_test['word'] + x_test["sep"] + x_test["sentence1"] + x_test["sep"] + x_test["sentence2"]
    x_test["Sent"] = x_test["sentence1"] + x_test["sep"] + x_test["sentence2"]
    x_test.to_csv('TestData.csv')

prepareTestDataset()

In [7]:
class TestDataSet(torch.utils.data.Dataset):
    def __init__(self, tokens, n):
        self.tokens = tokens
        self.len = n

    def __getitem__(self, idx):
        #format of dataset getitem as specified on Hugging Face
        item = {k: torch.tensor(v[idx]) for k, v in self.tokens.items()}
        return item

    def __len__(self):
        return self.len

In [8]:
#Loading test data
test_df = pd.read_csv("/content/TestData.csv",encoding = 'latin-1')
x_test1 = test_df["sentence1"].values.tolist()
x_test2 = test_df['sentence2'].values.tolist()
n = len(x_test1)

#Prep for model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
x_test_tok = tokenizer(x_test1,x_test2,padding="max_length",max_length = 128, truncation = True)
test_dataset = TestDataSet(x_test_tok,n)

#loading my best model saved in gdrive
best_model = '/content/content/models/checkpoint-1358'
testmodel = BertForSequenceClassification.from_pretrained(best_model, num_labels=2)
train_test = Trainer(model = testmodel)
pred_probs,_,_ = train_test.predict(test_dataset)
y_pred = np.argmax(pred_probs,axis=1)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

***** Running Prediction *****
  Num examples = 638
  Batch size = 8


In [9]:
#saving output predictions in the correct format
output = pd.Series(y_pred) 
output = output.apply(lambda x: 'F' if x == 0 else 'T')
output.to_csv("output.txt", header=None, index=None)

In [10]:
## DONT CHANGE THIS CELL
# Your testing code must produce a file output.txt with predictions as T and F in each line

## Final Evaluation 
# this is currently the same as dev.gold.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
correct, total = 0., 0.
for prediction, gold in zip(open('output.txt'), open('test.gold.txt')):
  prediction, gold = prediction.strip(), gold.strip()
  total += 1
  if prediction == gold:
    correct += 1

## Report this as the final validation performance 
print('Performance = ', (correct/total))

--2021-10-25 18:06:11--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt [following]
--2021-10-25 18:06:12--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1276 (1.2K) [text/plain]
Saving to: ‘test.gold.txt’

test.gold.txt       100%[===================>]   1.25K  --.-KB/s    in 0s      

2021-10-25 18:06:13 (53.3 MB/s) - ‘test.gold.txt’ saved [1276/1276]

Performance =  0.7163009404388715
